# Positioner Constants Update

Utilities for updating the positioner constants.

References:
 - [Constants viewer](https://replicator.desi.lbl.gov/ConstantsDB/app)
 - [Constants graphical browser](https://observablehq.com/@dkirkby/desi-pos-const)
 - [ConstantsDB wiki page](https://desi.lbl.gov/trac/wiki/FPS/ConstDB)

In [1]:
import requests
import json

In [2]:
import pandas as pd

Read constants from the DESI mirror:

In [3]:
def get_constants(name, version):
    url = f'https://replicator.desi.lbl.gov/ConstantsDB/app/Groups/exportJSON?name={name}&version={version}'
    r = requests.get(url)
    r.raise_for_status()
    return r.json()

Lookup the latest version of the `fiber_positioner` constants group to load [here](https://replicator.desi.lbl.gov/ConstantsDB/app/Groups/list?name=fiber_positioner).

In [4]:
constants = get_constants('fiber_positioner', 198)

Load a local copy of the current positioner index table listed [here](https://desi.lbl.gov/trac/browser/code/online/Positioner/PositionerIndexTable/trunk/index_files?order=date&desc=1):

In [5]:
index = pd.read_csv('desi_positioner_indexes_20211005.csv')

## 20220319 Update

This update addresses two issues:
 - P3 duty cycles are all set to 100%.  Change these to 70%.
 - [Ticket 964](https://desi.lbl.gov/trac/ticket/964) A few functional robots have backlash compensation disabled. Enable it for all robots (including non-functional ones).

In [7]:
def update_20220319(constants=constants, index=index, save='const_20220319.json'):
    
    output = []

    nbacklash = 0
    backlash_keys = dict(ANTIBACKLASH_ON=True, FINAL_CREEP_ON=True, MIN_DIST_AT_CRUISE_SPEED=180.0)

    ndutycycle = 0
    dutycycle_keys = dict(CURR_CREEP=70, CURR_CRUISE=70, CURR_SPIN_UP_DOWN=70)
    
    for i, element in enumerate(constants['elements']):
        
        name = element['name']
        info = index[index.DEVICE_ID == name]
        if len(info) == 0:
            print(f'Skipping {name} with no index entry')
            continue
        info = info.iloc[0]
        constants = dict(element['constants'])
        changed = False
        
        no_backlash = [constants[k] != v for k,v in backlash_keys.items()]
        if any(no_backlash):
            assert all(no_backlash)
            for k,v in backlash_keys.items():
                constants[k] = v
            nbacklash += 1
            changed = True
            
        if info.PETAL_LOC == 3:
            duty100 = [constants[k] != v for k,v in dutycycle_keys.items()]
            assert all(duty100)
            for k,v in dutycycle_keys.items():
                constants[k] = v
            ndutycycle += 1
            changed = True
            
        if changed:
            output.append(dict(name=name, constants=constants))
            
    print(f'Enabling backlash compensation for {nbacklash} robots')
    print(f'Setting 70% dutycycle for {ndutycycle} P3 robots')
    print(f'Updated constants for {len(output)} robots')
    
    with open(save, 'w') as f:
        json.dump(output, f, indent=4)

update_20220319()

Skipping M01722 with no index entry
Skipping M02725 with no index entry
Skipping M03236 with no index entry
Skipping M03556 with no index entry
Skipping M03567 with no index entry
Skipping M03648 with no index entry
Skipping M03709 with no index entry
Skipping M03824 with no index entry
Skipping M03912 with no index entry
Skipping M03918 with no index entry
Skipping M04024 with no index entry
Skipping M04182 with no index entry
Skipping M05743 with no index entry
Skipping M06848 with no index entry
Skipping M06891 with no index entry
Skipping M06931 with no index entry
Skipping M07550 with no index entry
Skipping M03996 with no index entry
Enabling backlash compensation for 441 robots
Setting 70% dutycycle for 502 P3 robots
Updated constants for 884 robots
